# laboratory work №3


In [1]:
from spyre import server

import pandas as pd
import os
import urllib3
import json
import re
import matplotlib.pyplot as plt

import __main__ as main
main.__file__ = "main_file"

## part 1 - prepare to app

- from lab2
+ Creating dataFrame with files from directory
+ Adding index through regular (from file's name)

In [2]:
def reading_text(directory):
    headers = ['Year', 'Week', 'SMN', 'SMT', 'VCI', 'TCI', 'VHI', 'empty']
    dfs = []

    # filenames = next(os.walk(directory), (None, None, []))[2] 
    filenames = os.listdir(directory)
    #print(filenames)
    for filename in filenames:
        # print(filename)
        if filename.startswith("NOAA") and filename.endswith(".csv"):
            
            j = int(re.search('\d+', filename).group())
            #print(j)

            file_path = os.path.join(directory, filename)
            df = pd.read_csv(file_path, header=1, names=headers)

            df = df.drop(df.loc[df['VHI'] == -1].index)
            df['area'] = j
            dfs.append(df)
            
            
    final_df = pd.concat(dfs, ignore_index=True)

    final_df = final_df.map(lambda x: re.sub(r'[^\d]', '', str(x)).strip() if isinstance(x, str) else x)
    final_df['Year'] = pd.to_numeric(final_df['Year'])
    final_df['Year'] = final_df['Year'].fillna(0).astype(int)
    
    # print(final_df.shape) 
        
    # print(final_df)
    # print(final_df.dtypes)

    return(final_df)



In [3]:
df = reading_text("../lab_2/NOAA_files/")

df.shape
df.dtypes

# print(df.columns) 
# print(df.index) 

Year       int64
Week     float64
SMN      float64
SMT      float64
VCI      float64
TCI      float64
VHI      float64
empty    float64
area       int64
dtype: object

### SimpleApp

+ web app with module Spyre
обрати часовий ряд VCI, TCI, VHI для набору даних із лабораторної
роботи 2 (dropdown список);

- Вибрати область, для якої буде виконуватись аналіз (dropdown список);
- Зазначити інтервал тижнів, за які відбираються дані;
- Зазначити інтервал років та продумати відображення на графіку обраних
даних (часові ряди задіапазон років, що обмежені інтервалом тижнів)
- Створити кілька вкладок для відображення таблиці із даними на графіку
ходу індексів;

In [4]:

class SimpleApp(server.App):
    title = "NOAA data visualization"
    inputs = [
            { "type": 'dropdown',
                    "label": "NOAA data dropdown",
                    "options" : [ {"label": "VCI", "value":"VCI" },
                        {"label":"TCI", "value":"TCI"},
                        {"label":"VHI", "value":"VHI"}],
                    "key": 'ticker',
                    "default": "VCI",
                    "action_id": "update_data"
            },
            {
                        "type": 'dropdown',
                        "label": "NOAA ",
                        "options" : [ {"label": "1: Cherkasy", "value":1 },
                            {"label":"2:Chernihiv", "value":2},
                            {"label":"3:Chernivtsi", "value":3},
                            {"label":"4:Crimea", "value":"4:Crimea"},
                            {"label":"5:Dnipropetrovs’k", "value":5},
                            {"label":"6:Donets’k", "value":6},
                            {"label":"7:Ivano-Frankivsk", "value":7},
                            {"label":"8:Kharkiv", "value":8},
                            {"label":"9:Kherson", "value":9},
                            {"label":"10:Khmelnytskyy", "value":10},
                            {"label":"11:Kiev", "value":11},
                            {"label":"12:Kiev City", "value":12},
                            {"label":"13:Kirovohrad", "value":13},
                            {"label":"14:Luhansk", "value":14},
                            {"label":"15:L’viv", "value":15},
                            {"label":"16:Mykolaiv", "value":16},
                            {"label":"17:Odessa", "value":17},
                            {"label":"18:Poltava", "value":18},
                            {"label":"19:Rivne", "value":19},
                            {"label":"20:Sevastopol’", "value":20},
                            {"label":"21:Sumy", "value":21},
                            {"label":"22:Ternopil", "value":22},
                            {"label":"23:Transcarpathia", "value":23},
                            {"label":"24:Vinnytsya", "value":24},
                            {"label":"25:Volyn", "value":25},
                            {"label":"26:Zaporizhzhya", "value":26},
                            {"label":"", "value":7}],
                        "key": 'area',
                        'default':26,
                        "action_id": "update_data" 
            },{
                            "type": 'slider',
                            "label": 'Indicate the year of the beginning of the interval',
                            "min": 1982, "max": 2024,
                            "value": 1990,
                            "default": 1921,
                            "key": 'year_first',
                            "action_id": 'update_data'
            },{
                            "type": 'slider',
                            "label": 'Indicate the year of the end  of the interval',
                            "min": 1982, "max": 2024,
                            "value": 2000,
                            "default": 1990,
                            "key": 'year_second',
                            "action_id": 'update_data'
            },{
                            "type": "text",
                            "label": "Specify the weekly interval",
                            "action_id": "update_data",
                            "value": "1,2",
                            "default": "1,2",
                            "key":"week"
            } 
                
        ]
    
    controls = [{   "control_type" : "hidden",  
                    "label" : "NOAA",
                    "control_id" : "update_data"}]
    
    tabs = ["Plot", "Table"]

    outputs = [
        {
            "type": "plot",
            "id": "plot",
            "control_id": "update_data",
            "tab": "Plot"},
        {
            "type": "table",
            "id": "table_id",
            "control_id": "update_data",
            "tab": "Table",
            "on_page_load": True
        }
    ]

    # df.head()

    def getData(self, params):
        
        ticker = params['ticker']
        area = int(params['area'])
        year_first = int(params['year_first'])
        year_second = int(params['year_second'])
        week = (params['week'].split(','))
        week_start = float(week[0].strip())  
        week_end = float(week[1].strip())
        
        # print("Total score for %s is %s" % (area, year_second ))
        # print(df['area'] == area)
        # print(type(area))

        final_df = df[(df['area'] == area) & 
                           (df['Year'] >= year_first) & (df['Year'] <= year_second) &
                          (df['Week'] >= week_start) & (df['Week'] <= week_end)]

        # print (final_df[ ticker])
        # print(ticker)
        return final_df[['Year', 'Week',ticker]]
    

    def getPlot(self, params):
        df_f = self.getData(params).set_index(['Year', 'Week'])
        print(111)
        print(df_f)
        obj = df_f.plot(figsize=(15, 6))
        fig = obj.get_figure()
        
        return fig


In [5]:
app = SimpleApp()
app.launch(port=8081)

[28/Oct/2024:19:54:03] ENGINE Listening for SIGTERM.
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Listening for SIGTERM.
[28/Oct/2024:19:54:03] ENGINE Listening for SIGHUP.
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Listening for SIGHUP.
[28/Oct/2024:19:54:03] ENGINE Listening for SIGUSR1.
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Listening for SIGUSR1.
[28/Oct/2024:19:54:03] ENGINE Bus STARTING
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Bus STARTING
CherryPy Checker:
The Application mounted at '' has an empty config.

[28/Oct/2024:19:54:03] ENGINE Started monitor thread 'Autoreloader'.
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Started monitor thread 'Autoreloader'.
[28/Oct/2024:19:54:03] ENGINE Serving on http://127.0.0.1:8081
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Serving on http://127.0.0.1:8081
[28/Oct/2024:19:54:03] ENGINE Bus STARTED
INFO:cherrypy.error:[28/Oct/2024:19:54:03] ENGINE Bus STARTED


127.0.0.1 - - [28/Oct/2024:19:54:09] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
111


INFO:cherrypy.access.4638282640:127.0.0.1 - - [28/Oct/2024:19:54:09] "GET /spinning_wheel HTTP/1.1" 200 2663 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


127.0.0.1 - - [28/Oct/2024:19:54:10] "GET /table?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&output_id=table_id& HTTP/1.1" 200 55296 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
             TCI
Year Week       
1994 1.0    4.31
     2.0    6.08
     3.0    9.09
     4.0    9.50
     5.0   15.27
...          ...
2007 48.0  70.08
     49.0  71.80
     50.0  74.09
     51.0  66.65
     52.0  55.75

[689 rows x 1 columns]


INFO:cherrypy.access.4638282640:127.0.0.1 - - [28/Oct/2024:19:54:10] "GET /table?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&output_id=table_id& HTTP/1.1" 200 55296 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
ERROR:root:Error: getPlot method must return an pyplot figure or matplotlib Axes object


127.0.0.1 - - [28/Oct/2024:19:54:10] "GET /plot?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&output_id=plot& HTTP/1.1" 200 96042 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"


INFO:cherrypy.access.4638282640:127.0.0.1 - - [28/Oct/2024:19:54:10] "GET /plot?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&output_id=plot& HTTP/1.1" 200 96042 "http://127.0.0.1:8081/?ticker=TCI&area=12&year_first=__float__1994&year_second=__float__2007&week=1,55&" "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/130.0.0.0 Safari/537.36"
